<a href="https://colab.research.google.com/github/ggaoshen/graph_wavenet/blob/sg-final/CS224W_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS224W - Final Project**

Description goes here.

# Device
You might need to use GPU for this Colab.

Please click `Runtime` and then `Change runtime type`. Then set the `hardware accelerator` to **GPU**.

# Installation

In [1]:
# Install torch geometric
import os
import torch
if 'IS_GRADESCOPE_ENV' not in os.environ:
  torch_version = str(torch.__version__)
  scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  !pip install torch-scatter -f $scatter_src
  !pip install torch-sparse -f $sparse_src
  !pip install torch-geometric
  !pip install -q git+https://github.com/snap-stanford/deepsnap.git
  !pip install -U -q PyDrive

Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 34.0 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !nvcc --version
  !python -c "import torch; print(torch.version.cuda)"

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
12.1


In [3]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  import torch
  print(torch.__version__)
  import torch_geometric
  print(torch_geometric.__version__)

2.1.0+cu121
2.4.0


In [4]:
# Import GraphWaveNet module
!git clone -b sg-experiment https://github.com/ggaoshen/graph_wavenet.git # NOTE: choose the right branch
%load graph_wavenet/src/graphwavenet.py
import sys
sys.path.append('graph_wavenet/src/')
from graphwavenet import GraphWaveNet
from util import masked_rmse, masked_mse, masked_mae, masked_mape, metric, temporal_dataset_split

Cloning into 'graph_wavenet'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 198 (delta 17), reused 14 (delta 8), pack-reused 152
Receiving objects: 100% (198/198), 163.30 KiB | 899.00 KiB/s, done.
Resolving deltas: 100% (104/104), done.


In [5]:
!pip install -U torch-geometric-temporal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 747.9 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 7.1 MB/s eta 0:00:00
  Created wheel for torch-geometric-temporal: filename=torch_geometric_temporal-0.54.0-py3-none-any.whl size=86722 sha256=91b8e0f09160c65b93a69f8fd17e6c950532f4c164038485c8181b93cf545c6d
  Stored in directory: /root/.cache/pip/wheels/9e/9b/b6/e15256e053f0cb49b1084a67a709db909d418386a231f0722c
Successfully built torch-geometric-temporal
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires pyt

In [6]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context



# Temporal Datasets

from torch_geometric_temporal.dataset import METRLADatasetLoader
loader = METRLADatasetLoader()
dataset = loader.get_dataset(num_timesteps_in=12, num_timesteps_out=12)
iterator = iter(dataset)
print("METRLA dataset from original Graph Wavenet paper: \n", next(iterator))


from torch_geometric_temporal.dataset import ChickenpoxDatasetLoader
dataset = ChickenpoxDatasetLoader().get_dataset(lags=8) # consistent with chickenpox paper
iterator = iter(dataset)
print("Chickenpox dataset: \n", next(iterator))


METRLA dataset from original Graph Wavenet paper: 
 Data(x=[207, 2, 12], edge_index=[2, 1722], edge_attr=[1722], y=[207, 12])
Chickenpox dataset: 
 Data(x=[20, 8], edge_index=[2, 102], edge_attr=[102], y=[20])


In [ ]:
# Run Chickenpox Dataset

import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd

# training
from tqdm import tqdm
import time

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

save_path = "store/checkpoint"
if not os.path.exists(save_path):
    os.makedirs(save_path)

# Get data

# from torch_geometric_temporal.signal import temporal_signal_split
train_ratio, val_ratio = 0.8, 0.1
# train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=train_ratio)
train_dataset, val_dataset, test_dataset = temporal_dataset_split(dataset, train_split = train_ratio, validation_split = val_ratio)
val_offset = int(dataset.snapshot_count * train_ratio) # starting index for valid set
test_offset = val_offset+int(dataset.snapshot_count * val_ratio) # starting index for valid set

def prepare_n_period_y(dataset):

    res = []
    for data in dataset:
        res.append(data.y)
    res = torch.stack(res, dim=0)

    return res

y_all = prepare_n_period_y(dataset)


# Model inputs
in_dim = dataset[0].num_node_features # 8 treat lagged inputs as node features
out_dim = 1
num_nodes = dataset[0].num_nodes # 1068
timesteps_to_predict = 10 # 10, 20, 40 week forecast horizon
epochs = 200
lrate = 0.0001
wdecay = 0.001

# early stopping parameters
patience = 10
counter = 0
best_val_loss = float('inf')


model = GraphWaveNet(
    num_nodes=num_nodes,
    in_channels=in_dim,
    out_channels=out_dim,
    out_timesteps=timesteps_to_predict,
).to(device)

# Training loop
print("start training...", flush=True)
his_loss = []
val_time = []
train_time = []
best_epoch = 0

optimizer = torch.optim.Adam(model.parameters(), lr=lrate, weight_decay=wdecay)

model.train()

training_curve_dict = {"epoch_train_loss": [], "epoch_valid_loss": []}

for epoch in tqdm(range(epochs)):
    train_loss = []
    t1 = time.time()

    for i, snapshot in enumerate(train_dataset):
        x_train = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)
        if timesteps_to_predict == 1:
            y_train = snapshot.y.to(device)
        else:
            y_train = y_all[i : i + timesteps_to_predict,:].to(device)

        pred = model(x_train, snapshot.edge_index, snapshot.edge_attr).squeeze()
        loss = masked_mse(pred, y_train, 0.0) # mean squared error for loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        train_loss.append(loss.item())

    mtrain_loss = np.mean(train_loss)
    training_curve_dict['epoch_train_loss'].append(mtrain_loss)
    print(f"training loss: {mtrain_loss}")


    t2 = time.time()
    train_time.append(t2 - t1)
    valid_loss = []

    s1 = time.time()
    for i, snapshot in enumerate(val_dataset):
        x_val = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)
        if timesteps_to_predict == 1:
            y_val = snapshot.y.to(device)
        else:
            y_val = y_all[val_offset + i : val_offset + i + timesteps_to_predict,:].to(device)

        with torch.no_grad():
            pred = model(x_val, snapshot.edge_index, snapshot.edge_attr).squeeze()
        loss = masked_mse(pred, y_val, 0.0).cpu().numpy()
        valid_loss.append(loss)

    s2 = time.time()
    # log = "Epoch: {:03d}, Inference Time: {:.4f} secs"
    # print(log.format(i, (s2 - s1)))
    val_time.append(s2 - s1)

    mvalid_loss = np.mean(valid_loss)
    training_curve_dict['epoch_valid_loss'].append(mvalid_loss)

    his_loss.append(mvalid_loss)

    if np.argmin(his_loss) == len(his_loss) - 1:
        torch.save(
            model.state_dict(), save_path + "/epoch_" + str(i) + ".pth"
        )
        best_epoch = i

    log = (
        "Epoch: {:03d}, Train Loss: {:.4f}, "
        + "Valid Loss: {:.4f}, "
        + "Training Time: {:.4f}/epoch"
    )
    print(
        log.format(
            i,
            mtrain_loss,
            mvalid_loss,
            (t2 - t1),
        ),
        flush=True,
    )
print("Average Training Time: {:.4f} secs/epoch".format(np.mean(train_time)))
print("Average Inference Time: {:.4f} secs".format(np.mean(val_time)))


# eval
model.load_state_dict(
    torch.load(save_path + "/epoch_" + str(best_epoch) + ".pth")
)
model.eval()
loss = 0
for i, snapshot in enumerate(test_dataset[:-timesteps_to_predict]):

    x_test = snapshot.x.reshape(-1, num_nodes, in_dim).to(device)

    if timesteps_to_predict == 1:
        y_test = snapshot.y.to(device)
    else:
        y_test = y_all[test_offset+i : test_offset+i + timesteps_to_predict,:].to(device)


    with torch.no_grad():
        pred = model(x_test, snapshot.edge_index, snapshot.edge_attr).squeeze()
    loss += masked_mse(pred, y_test, 0.0) # mean squared error as loss

loss = loss / (i+1)
loss = loss.item()
print("Test MSE Loss: {:.4f}".format(loss))


# store training loss
df = pd.DataFrame(training_curve_dict)
df.to_csv("training_curve.csv")

from google.colab import files
files.download('training_curve.csv')

# plot training and validation loss
df.plot()


start training...


  0%|          | 0/200 [00:00<?, ?it/s]

training loss: 0.9670065093149499
Epoch: 050, Train Loss: 0.9670, Valid Loss: 0.8954, Training Time: 21.9304/epoch


  0%|          | 1/200 [00:23<1:18:56, 23.80s/it]

training loss: 0.9522934666039741
Epoch: 050, Train Loss: 0.9523, Valid Loss: 0.8916, Training Time: 22.5046/epoch


...

  8%|▊         | 15/200 [06:04<1:15:37, 24.53s/it]

training loss: 0.9302871889548331
Epoch: 050, Train Loss: 0.9303, Valid Loss: 0.8853, Training Time: 24.0991/epoch


  8%|▊         | 16/200 [06:30<1:16:05, 24.81s/it]

training loss: 0.9300256919252073
Epoch: 050, Train Loss: 0.9300, Valid Loss: 0.8848, Training Time: 24.1099/epoch


  8%|▊         | 17/200 [06:55<1:16:19, 25.03s/it]

In [9]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [10]:
# Reset GPU RAM if GPU out of memory

# !pip install numba

# from numba import cuda

# cuda.select_device(0) # choosing second GPU

# cuda.close()